In [1]:
from datetime import datetime
import os
import re
import glob
import requests 
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np

In [3]:
link = 'https://www.worldometers.info/coronavirus/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [4]:
thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

In [6]:
head_rows = []
body_rows = []
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [9]:
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         
df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"2,648,891","+38,601","151,822","+1,447","1,181,457","+29,113","1,315,612","19,335",,,,,,North America,\n,,
1,,\nSouth America\n,"1,689,318","+61,065","68,554","+1,630","913,769","+27,766","706,995","12,378",,,,,,South America,\n,,
2,,\nEurope\n,"2,283,078","+15,341","186,934",+523,"1,235,008","+15,875","861,136","6,370",,,,,,Europe,\n,,
3,,\nAsia\n,"1,816,317","+41,015","46,363",+908,"1,143,152","+28,628","626,802","16,196",,,,,,Asia,\n,,
4,,\nAfrica\n,"288,715","+10,038","7,711",+261,"135,081","+6,734","145,923",505,,,,,,Africa,\n,,


In [10]:
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)
df_bs = df_bs.drop('#', axis=1)
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"2,294,247","+30,596","121,380",+692,"955,291","+24,297","1,217,576","16,526","6,933",367,"27,262,593","82,380","330,938,757",North America
1,Brazil,"1,032,913","+49,554","48,954","+1,085","520,360","+16,853","463,599","8,318","4,861",230,"2,344,437","11,032","212,508,714",South America
2,Russia,"569,063","+7,972","7,841",+181,"324,406","+10,443","236,816","2,300","3,899",54,"16,321,964","111,846","145,932,574",Europe
3,India,"395,812","+14,721","12,970",+366,"214,206","+9,024","168,636","8,944",287,9,"6,426,627","4,659","1,379,530,022",Asia
4,UK,"301,815","+1,346","42,461",+173,N/A,N/A,N/A,379,"4,447",626,"7,433,114","109,513","67,874,383",Europe


In [11]:
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']
df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"330,938,757","2,294,247","+30,596","121,380",+692,"955,291","+24,297","1,217,576","16,526","6,933",367,"27,262,593","82,380"
1,Brazil,South America,"212,508,714","1,032,913","+49,554","48,954","+1,085","520,360","+16,853","463,599","8,318","4,861",230,"2,344,437","11,032"
2,Russia,Europe,"145,932,574","569,063","+7,972","7,841",+181,"324,406","+10,443","236,816","2,300","3,899",54,"16,321,964","111,846"
3,India,Asia,"1,379,530,022","395,812","+14,721","12,970",+366,"214,206","+9,024","168,636","8,944",287,9,"6,426,627","4,659"
4,UK,Europe,"67,874,383","301,815","+1,346","42,461",+173,N/A,N/A,N/A,379,"4,447",626,"7,433,114","109,513"


In [13]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [14]:
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['CAR', 'Mayotte', 'French Guiana', 'Diamond Princess',
       'Channel Islands', 'Réunion', 'Isle of Man', 'Martinique',
       'Cayman Islands', 'Faeroe Islands', 'Gibraltar', 'Guadeloupe',
       'Brunei ', 'Aruba', 'Sint Maarten', 'French Polynesia', 'Macao',
       'Saint Martin', 'St. Vincent Grenadines', 'Curaçao',
       'New Caledonia', 'Falkland Islands', 'Turks and Caicos',
       'Montserrat', 'MS Zaandam', 'British Virgin Islands'], dtype=object)

In [15]:
for col in df_bs.columns[2:]:
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)
df_bs = df_bs.replace('', np.nan)
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,330938757,2294247,30596,121380,692,955291,24297,1217576,16526,6933,367,27262593,82380,Americas
1,Brazil,South America,212508714,1032913,49554,48954,1085,520360,16853,463599,8318,4861,230,2344437,11032,Americas
2,Russia,Europe,145932574,569063,7972,7841,181,324406,10443,236816,2300,3899,54,16321964,111846,Europe
3,India,Asia,1379530022,395812,14721,12970,366,214206,9024,168636,8944,287,9,6426627,4659,South-EastAsia
4,UK,Europe,67874383,301815,1346,42461,173,NaN,NaN,NaN,379,4447,626,7433114,109513,Europe


In [16]:
df_bs.to_csv('worldometer_data.csv', index=False)